# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [11]:
!pip install cartopy
!pip install geoviews
!pip install pyproj

In [12]:
# Dependencies and Setup
import pandas as pd
import hvplot.pandas
import requests
import json

# Import API key
from api_keys import geoapify_key

In [13]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,chibougamau,49.9168,-74.3659,11.82,79,100,3.44,CA,1701900060
1,1,port-aux-francais,-49.3500,70.2167,40.08,91,100,21.00,TF,1701900061
2,2,margaret river,-33.9500,115.0667,57.74,82,99,12.33,AU,1701900062
3,3,eldorado at santa fe,35.5264,-105.9347,55.31,35,0,8.99,US,1701900063
4,4,puerto natales,-51.7236,-72.4875,55.85,44,20,24.16,CL,1701900064


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [14]:
%%capture --no-display

# Configure the map plot
city_plot = city_data_df.hvplot.points(
    'Lng',
    'Lat',
    geo=True, 
    tiles = 'OSM', 
    frame_width = 700, 
    frame_height = 500, 
    color='City',
    alpha= 0.5,
    s='Humidity',
    scale= 0.8)

# Display the map
city_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [15]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_df = city_data_df.loc[(city_data_df['Max Temp']<=79)\
                            &(city_data_df['Max Temp']>=62)\
                            &(city_data_df['Cloudiness']<=60)\
                            &(city_data_df['Country']!='IN')\
                            &(city_data_df['Country']!='CN')\
                            &(city_data_df['Country']!='RU')\
                            &(city_data_df['Country']!='PK')
                            &(city_data_df['Humidity']<50),:]

# Drop any rows with null values
ideal_df.dropna()

# Display sample data
print(ideal_df)

     City_ID            City      Lat       Lng  Max Temp  Humidity  \
156      156        fiambala -27.6871  -67.6149     75.00        42   
168      168         diapaga  12.0708    1.7889     78.46        14   
170      170        calvinia -31.4707   19.7760     70.54        27   
182      182          bardai  11.6500    7.2333     70.48        18   
257      257          austin  30.2672  -97.7431     70.66        37   
316      316        bougouni  11.4170   -7.4790     78.12        18   
323      323         salalah  17.0151   54.0924     77.09        41   
325      325  guerrero negro  27.9769 -114.0611     76.87        30   
357      357          agadez  19.7500   10.2500     64.15        25   
359      359         dongola  19.1667   30.4833     66.51        26   
361      361       el fasher  13.6279   25.3494     66.99        13   
368      368    stephenville  32.2207  -98.2023     68.86        36   
392      392            kita  13.0349   -9.4895     78.19        17   
400   

### Step 3: Create a new DataFrame called `hotel_df`.

In [16]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_df.iloc[:,[1,8,2,3,5]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name']=''

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
156,fiambala,AR,-27.6871,-67.6149,42,
168,diapaga,BF,12.0708,1.7889,14,
170,calvinia,ZA,-31.4707,19.7760,27,
182,bardai,NG,11.6500,7.2333,18,
257,austin,US,30.2672,-97.7431,37,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [17]:
# Set parameters to search for a hotel
radius = 10000
params = {'categories': 'accommodation.hotel', 'filter':'', 'bias':'', 'apiKey': geoapify_key}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lon = row["Lng"]
    lat = row["Lat"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f'circle:{lon},{lat},{radius}'
    params["bias"] = f'proximity:{lon},{lat}'
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search


fiambala - nearest hotel: Cabañas Lunita
diapaga - nearest hotel: Le Gacilien
calvinia - nearest hotel: No hotel found
bardai - nearest hotel: No hotel found
austin - nearest hotel: The Driskill
bougouni - nearest hotel: Hotel Piemont
salalah - nearest hotel: Muscat International Hotel
guerrero negro - nearest hotel: Plaza sal paraiso
agadez - nearest hotel: No hotel found
dongola - nearest hotel: Lord
el fasher - nearest hotel: Noor Aleman
stephenville - nearest hotel: No hotel found
kita - nearest hotel: Hotel Chats Rouges
canyon - nearest hotel: Buffalo Inn
khasab - nearest hotel: "Extra Divers Villa"
abu jibeha - nearest hotel: No hotel found
el campo - nearest hotel: Days Inn by Wyndham El Campo TX
kutum - nearest hotel: No hotel found
las vegas - nearest hotel: Downtown Grand
north platte - nearest hotel: Howard Johnson Inn - North Platte
menaka - nearest hotel: No hotel found
adelia maria - nearest hotel: Hostal Los Olmos


,City,Country,Lat,Lng,Humidity,Hotel Name
156,fiambala,AR,-27.6871,-67.6149,42,Cabañas Lunita
168,diapaga,BF,12.0708,1.7889,14,Le Gacilien
170,calvinia,ZA,-31.4707,19.7760,27,No hotel found
182,bardai,NG,11.6500,7.2333,18,No hotel found
257,austin,US,30.2672,-97.7431,37,The Driskill
316,bougouni,ML,11.4170,-7.4790,18,Hotel Piemont
323,salalah,OM,17.0151,54.0924,41,Muscat International Hotel
325,guerrero negro,MX,27.9769,-114.0611,30,Plaza sal paraiso
357,agadez,NE,19.7500,10.2500,25,No hotel found
359,dongola,SD,19.1667,30.4833,26,Lord


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [20]:
%%capture --no-display

# Configure the map plot
hotel_plot = hotel_df.hvplot.points('Lng',
            'Lat',
            geo=True, 
            tiles = 'OSM', 
            frame_width = 700, 
            frame_height = 500, 
            color='City',
            alpha=0.6,
            hover=True,
            hover_cols='all',
            use_index=False,
            s='Humidity')

# Display the map
hotel_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)